In [1]:
import torch
from torch_geometric.datasets import MovieLens1M, MovieLens100K


# Load MovieLens 1M dataset (adjust root if needed)
#dataset = MovieLens1M(root='/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg')
dataset= MovieLens100K(root="/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg")
print(len(dataset))


/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1


In [4]:
graph = dataset[0]
dataset[0].num_nodes
print (dataset.data)
print (graph["user"].x[:10,:])
print (graph["movie"].x[:10,:])


HeteroData(
  movie={ x=[1682, 18] },
  user={ x=[943, 24] },
  (user, rates, movie)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
    edge_label_index=[2, 20000],
    edge_label=[20000],
  },
  (movie, rated_by, user)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  }
)
tensor([[0.3288, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000],
        [0.7260, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3151, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.3288, 0.0000, 1.

In [7]:

movies_features = graph["movie"].x
print (movies_features[:4])

tensor([[0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [8]:

user_features = graph["user"].x
print (movies_features[:10])

tensor([[0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])


In [8]:
import os
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 

In [ ]:
import os
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
%run script/run.py -c config/transductive/inference.yaml --dataset WN18RR --epochs 0 --bpe null --gpus [0] --ckpt /usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth

In [1]:
%run script/run.py -c config/transductive/inference.yaml --dataset MovieLens100k --epochs 0 --bpe null --gpus "[0]" --ckpt /usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
12:16:12   Random seed: 1024
12:16:12   Config file: config/transductive/inference.yaml
12:16:12   {'checkpoint': '/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth',
 'dataset': {'class': 'MovieLens100k', 'root': '~/git/ULTRA/kg-datasets/'},
 'model': {'class': 'Ultra',
           'entity_model': {'aggregate_func': 'sum',
                            'class': 'EntityNBFNet',
                            'hidden_dims': [64, 64, 64, 64, 64, 64],
                            'input_dim': 64,
                            'layer_norm': True,
                            'message_func': 'distmult',
                            'short_cut': True},
           'relation_model': {'aggregate_func':

/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ultra/tasks.py:194: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  Ahh = torch.sparse.mm(EhT, Eh).coalesce()
12:16:16   MovieLens100k dataset
12:16:16   #train: 64000, #valid: 8000, #test: 8000


rel graph is built


12:16:17   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
12:16:17   Evaluate on valid


Load rspmm extension. This may take a while...


12:16:20   Note: NumExpr detected 28 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
12:16:20   NumExpr defaulting to 8 threads.
12:17:08   mr: 204.833
12:17:08   mrr: 0.0520594
12:17:08   hits@1: 0.0203125
12:17:08   hits@3: 0.0455
12:17:08   hits@10: 0.107125
12:17:08   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
12:17:08   Evaluate on test
12:17:58   mr: 201.8
12:17:58   mrr: 0.052182
12:17:58   hits@1: 0.0205625
12:17:58   hits@3: 0.04525
12:17:58   hits@10: 0.107


In [ ]:
#train
%run script/run.py -c config/transductive/inference.yaml --dataset MovieLens100k --epochs 10 --bpe 100 --gpus "[0]" --ckpt /usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth

In [1]:
import numpy as np
import torch

print(np.__version__)  # Check NumPy version
print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Check if CUDA is available


1.24.3
2.1.0
True


In [2]:
print(os.getcwd())
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
print(os.getcwd())

NameError: name 'os' is not defined

In [13]:
print(os.listdir())

['valid.txt', 'requirements.txt', 'MovieLense_load.ipynb', 'temp_pyg', 'config', '.git', '.gitignore', 'ultra', 'LICENSE', 'script', 'test.txt', 'train.txt', 'ckpts', 'asset', 'README.md', '.ipynb_checkpoints']


In [1]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pitri (pitri-eth-z-rich). Use `wandb login --relogin` to force relogin


acc,▂▁▇▆▆▇█▇
loss,█▃▃▃▄▃▃▁
acc,0.83106
loss,0.0692
